# Book Market Analysis with SQL

There is given a database of one of the services competing in the book market. It contains data on books, publishers, authors, customer ratings and reviews of books. This information will be used to generate a value proposition for a new product from the area of apps for book lovers.

# Table of Contents

- 1) [Preparation](#1)
<br> <br>
    - 1.1) [Create engine and connection to database](#1.1)
    - 1.2) [Table 'books'](#1.2)
    - 1.3) [Table 'authors'](#1.3)
    - 1.4) [Table 'ratings](#1.4)
    - 1.5) [Table 'reviews'](#1.5)
    - 1.6) [Table 'publishers'](#1.6)
    - 1.7) [Conclusion](#1.7)
    <br> <br>
- 2) [Analysis](#2)
<br> <br>
    - 2.1) [Number of books released after January 1, 2000](#2.1)
    - 2.2) [The number of user reviews and the average rating for each book](#2.2)
    - 2.3) [Identify the publisher that has released the greatest number of books with more than 50 pages](#2.3)
    - 2.4) [Identify the author with the highest average book rating (look only at books with at least 50 ratings)](#2.4)
    - 2.5) [The average number of text reviews among users who rated more than 50 books](#2.5)
    <br> <br>
- 3) [Overall Conclusion](#3)

<a id="1"></a>

## 1) Preparation

<a id="1.1"></a>

### 1.1) Create engine and connection to database

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
# connection to database
db_config = {'user': 'user_name',         # user name
             'pwd': 'password', # password
             'host': 'host',
             'port': 'port',              # connection port
             'db': 'database'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# define function that gives the result of the SQL query
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

For that project we focus on the tables 'books', 'authors', 'ratings', 'reviews' and 'publishers'.

<a id="1.2"></a>

### 1.2) Table 'books'

In [4]:
# first five rows of table 'books'
query = '''
    SELECT
        *
    FROM 
        books
    LIMIT
        5;
'''

queryResult(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [ ]:
# length of table 'books'
query = '''
    SELECT
        COUNT(*) AS number_of_rows
    FROM 
        books;
'''

queryResult(query)

In [6]:
# is every book ID unique?
query = '''
    SELECT
        COUNT(DISTINCT book_id) AS number_of_book_ids
    FROM 
        books;
'''

queryResult(query)

,number_of_book_ids
0,1000


In [7]:
# is every book title unique?
query = '''
    SELECT
        COUNT(DISTINCT title) AS number_of_unique_titles
    FROM 
        books;
'''

queryResult(query)

,number_of_unique_titles
0,999


In [8]:
# show the title given twice in the table
query = '''
    SELECT
        title,
        COUNT(title) AS number_of_times_title_is_listed
    FROM 
        books
    GROUP BY
        title
    ORDER BY
        number_of_times_title_is_listed DESC
    LIMIT
        4;
'''

queryResult(query)

,title,number_of_times_title_is_listed
0,Memoirs of a Geisha,2
1,The Botany of Desire: A Plant's-Eye View of th...,1
2,The Poisonwood Bible,1
3,Count Zero (Sprawl #2),1


In [9]:
# see if the title 'Memoirs of a Geisha' is duplicated
query = '''
    SELECT
        *
    FROM 
        books
    WHERE
        title = 'Memoirs of a Geisha';
'''

queryResult(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Since the title above seems to be published two times from two different publishers within 7 days of difference, there is no reason to assume a duplicate, so the rows won't get deleted.

<a id="1.3"></a>

### 1.3) Table 'authors'

In [10]:
# first five rows of table 'authors'
query = '''
    SELECT
        *
    FROM 
        authors
    LIMIT
        5;
'''

queryResult(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [11]:
# length of table 'authors'
query = '''
    SELECT
        COUNT(*) AS number_of_rows
    FROM 
        authors;
'''

queryResult(query)

,number_of_rows
0,636


In [12]:
# is every author ID unique?
query = '''
    SELECT
        COUNT(DISTINCT author_id) AS number_of_author_ids
    FROM 
        authors;
'''

queryResult(query)

,number_of_author_ids
0,636


<a id="1.4"></a>

### 1.4) Table 'ratings'

In [13]:
# first five rows of table 'ratings'
query = '''
    SELECT
        *
    FROM 
        ratings
    LIMIT
        5;
'''

queryResult(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [14]:
# length of table 'ratings'
query = '''
    SELECT
        COUNT(*) AS number_of_rows
    FROM 
        ratings;
'''

queryResult(query)

,number_of_rows
0,6456


In [15]:
# is every rating ID unique?
query = '''
    SELECT
        COUNT(DISTINCT rating_id) AS number_of_rating_ids
    FROM 
        ratings;
'''

queryResult(query)

,number_of_rating_ids
0,6456


In [16]:
# maximum and minimum values for ratings
query = '''
    SELECT
        MIN(rating) AS minimum_rating,
        MAX(rating) AS maximum_rating
    FROM 
        ratings;
'''

queryResult(query)

,minimum_rating,maximum_rating
0,1,5


<a id="1.5"></a>

### 1.5) Table 'reviews'

In [17]:
# first five rows of table 'reviews'
query = '''
    SELECT
        *
    FROM 
        reviews
    LIMIT
        5;
'''

queryResult(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [18]:
# length of table 'reviews'
query = '''
    SELECT
        COUNT(*) AS number_of_rows
    FROM 
        reviews;
'''

queryResult(query)

,number_of_rows
0,2793


In [19]:
# is every review ID unique?
query = '''
    SELECT
        COUNT(DISTINCT review_id) AS number_of_review_ids
    FROM 
        reviews;
'''

queryResult(query)

,number_of_review_ids
0,2793


<a id="1.6"></a>

### 1.6) Table 'publishers'

In [20]:
# first five rows of table 'publishers'
query = '''
    SELECT
        *
    FROM 
        publishers
    LIMIT
        5;
'''

queryResult(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [21]:
# length of table 'publishers'
query = '''
    SELECT
        COUNT(*) AS number_of_rows
    FROM 
        publishers;
'''

queryResult(query)

,number_of_rows
0,340


In [22]:
# is every publisher ID unique?
query = '''
    SELECT
        COUNT(DISTINCT publisher_id) AS number_of_publisher_ids
    FROM 
        publishers;
'''

queryResult(query)

,number_of_publisher_ids
0,340


<a id="1.7"></a>

### 1.7) Conclusion

The table 'books' has exactly 1000 rows and it contains certain fields that are connected with the other tables: The column 'book_id' is also given in the tables 'ratings' (6456 rows) and 'reviews' (2793 rows). These two tables contain ratings (in numbers between 1 to 5) or reviews from certain people (username) for certain books (book ID). The table 'authors' (636 rows) is connected by the column 'author_id' with the 'books' table. The table 'publishers' (340 rows) is connected by the column 'publisher_id' with the 'books' table. For every table, all the IDs are unique.

<a id="2"></a>

## 2) Analysis

<a id="2.1"></a>

### 2.1) Number of books released after January 1, 2000

In [23]:
# Count the number of book IDs that have a publication date that is later than the 1st of January 2000

query = '''
    SELECT
        COUNT(DISTINCT book_id) AS number_of_books
    FROM 
        books
    WHERE
        publication_date :: date > '2000-01-01';
'''

queryResult(query)

,number_of_books
0,819


#### Conclusion

The number of books released after the 1st of January 2000 is 819 books. <br> Since the table has 1000 books, that means that around 18% of all books (= (1000-819)/1000) in the table 'books' have been published in the 90s or earlier.

<a id="2.2"></a>

### 2.2) The number of user reviews and the average rating for each book

In [24]:
# In the subquery I create a table that contains all ratings for every book ID; every of these ratings is given as often in the
# table as many reviews are given for that book ID. The number of reviews is given by the number of review IDs and the
# average rating is given by the average number of the ratings (the fact that all ratings are listed multiple times does not
# affect the average value for each book ID)

query = '''
    SELECT
        Sub.book_id,
        Sub.title,
        AVG(Sub.rating) AS avg_rating,
        COUNT(Sub.review_id) AS number_of_user_reviews
    FROM
        (SELECT
            books.book_id,
            books.title,
            ratings.rating,
            reviews.review_id
        FROM
            books
            INNER JOIN ratings ON books.book_id = ratings.book_id
            INNER JOIN reviews ON books.book_id = reviews.book_id) AS Sub
    GROUP BY
        Sub.book_id,
        Sub.title
    ORDER BY
        number_of_user_reviews DESC
    LIMIT
        5;
'''

queryResult(query)

,book_id,title,avg_rating,number_of_user_reviews
0,948,Twilight (Twilight #1),3.662500,1120
1,750,The Hobbit or There and Back Again,4.125000,528
2,673,The Catcher in the Rye,3.825581,516
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,492
4,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,480


#### Conclusion

For every book I calculated the average rating and the number of user reviews. The book with the most user reviews has been reviewed 7 times and has the book ID 948 with the title 'Twilight #1'. Comparing this book with the Harry Potter books in the table above, one can see that books with similar number of user reviews can have different average user ratings. That means that popular books don't necessarily need to be the best rated ones. One can assume that books with many user reviews are popular, so they have also many user ratings. That leads to a more solid average rating value that mirrors the opinions of many book lovers; an average rating of 5.0 is not solid if it is the average of one or two ratings.

<a id="2.3"></a>

### 2.3) Identify the publisher that has released the greatest number of books with more than 50 pages

In [25]:
# In the subquery I connect the publisher ID from the table 'books' with the publisher's name from the table 'publishers' and
# I set the condition to have only books with more than 50 pages. In the query, for every publisher I count the number of books
# and order by that number to see the five publishers with the greatest number of released books

query = '''
    SELECT
        Sub.publisher_id,
        Sub.publisher,
        COUNT(DISTINCT Sub.book_id) AS number_of_books
    FROM
        (SELECT
            books.book_id,
            books.num_pages,
            books.publisher_id,
            publishers.publisher
        FROM
            books
            INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
        WHERE
            books.num_pages > 50) AS Sub
    GROUP BY
        Sub.publisher_id,
        Sub.publisher
    ORDER BY
        number_of_books DESC
    LIMIT
        5;
'''

queryResult(query)

,publisher_id,publisher,number_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19


#### Conclusion

From the data given we can conclude: The publisher that has released the greatest number of books with more than 50 pages has the publisher ID 212, is named 'Penguin Books' and released 42 books with more than 50 pages. The condition of the pages excludes short brochures and similar publications. Looking at the big differences in the number of books from the biggest 5 publishers above one can derive that the book publishing market is leaded by only few publishers having the most books published. The difference between the biggest and the second biggest publisher is 11 books which corresponds to around 26% (= 11/42) from the 42 books published by 'Penguin Books'.

<a id="2.4"></a>

### 2.4) Identify the author with the highest average book rating (look only at books with at least 50 ratings)

In [26]:
# In the subquery, for every book I count the number of ratings and calculate the average rating. I choose only those books with
# at least 50 ratings. In the query, I group by authors and calculate the average book rating for every author. I order in 
# descending order of average book ratings and print the first five rows to see the five authors with the highest values

query = '''
    SELECT
        Sub.author_id,
        Sub.author,
        AVG(Sub.avg_rating_of_book) AS avg_book_rating_by_author
    FROM
        (SELECT
            books.book_id,
            books.author_id,
            authors.author,
            COUNT(ratings.rating),
            AVG(ratings.rating) AS avg_rating_of_book
        FROM
            books
            INNER JOIN authors ON books.author_id = authors.author_id
            INNER JOIN ratings ON books.book_id = ratings.book_id 
        GROUP BY
            books.book_id,
            books.author_id,
            authors.author
        HAVING
            COUNT(ratings.rating) >= 50) AS Sub
    GROUP BY
        Sub.author_id,
        Sub.author
    ORDER BY
        avg_book_rating_by_author DESC
    LIMIT
        3;
'''

queryResult(query)

,author_id,author,avg_book_rating_by_author
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446


#### Conclusion

The highest value for average book ratings is 4.28 and is achieved by the author J.K. Rowling; the author known for the Harry Potter books. The second name Mary GrandPré is an american illustrator for covers and chapters of the Harry Potter books. Looking at the top three authors one can see that their average book ratings range between 4.258 and 4.284 which is a small area; that means there are multiple popular authors having high book ratings.

<a id="2.5"></a>

### 2.5) The average number of text reviews among users who rated more than 50 books

In [27]:
# In the subquery I take the 'ratings' table and add data from the 'reviews' table to it in the following way: For every
# username from the 'ratings' table I count the number of unique reviews and count the number of unique books being rated;
# I only leave the data of usernames who rated more than 50 books. In the query I divide the total number of text reviews
# by the number of users to get the average number of text reviews

query = '''
    SELECT
        (SUM(Sub.number_of_text_reviews) / COUNT(DISTINCT Sub.username)) AS avg_number_of_text_reviews
    FROM
        (SELECT
            ratings.username,
            COUNT(DISTINCT(reviews.review_id)) AS number_of_text_reviews,
            COUNT(DISTINCT(ratings.book_id)) AS number_of_rated_books
        FROM
            ratings
            LEFT JOIN reviews ON ratings.username = reviews.username
        GROUP BY
            ratings.username
        HAVING
            COUNT(DISTINCT(ratings.book_id)) > 50) AS Sub;
'''

queryResult(query)

,avg_number_of_text_reviews
0,24.333333


#### Conclusion

Among those users who rated more than 50 books the average number of text reviews is around 24 text reviews per user. That means that user who love rating books with an number also like writing text reviews about them; but more rare since 24 text reviews is a much smaller number than 50 or more ratings. 

<a id="3"></a>

## 3) Overall Conclusion

The analysis gives us the following findings:

- Over 80% of all books have been published in the 2000s or later
- Popular books don't necessarily need to be the best rated ones
- Books with many user reviews are popular, so it's more likely that they also have many user ratings which lead to solid ratings
- The leader in the book publishing market is 'Penguin Books' having 42 of 1000 books published from the given data
- The book publishing market is leaded by only few publishers having the most books published
- The highest value for average book ratings is 4.28 and is achieved by the author J.K. Rowling; the author known for the Harry Potter books
- There are multiple popular authors having high book ratings
- Book lovers rating many books also likely do many text reviews about books

For a new product (like a new app for book lovers) in the book market, in generell it's important to think of the function of the product. It should focus on popular books since they have many ratings which give a better picture on how much readers enjoyed the book. It might be a good idea to have deals with certain publishers as there are certain publishers with a high share of books in the market. Having many books attracts book lovers since they love rating books and writting many reviews about them; even though if the book hasn't been rated high.